<a href="https://colab.research.google.com/github/sum-coderepo/Git-Test/blob/main/SentenceEncoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import gc
import os
import json
from collections import Counter, defaultdict
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import re
import csv  
year_pattern = r'([1-2][0-9]{3})'
pd.set_option('display.max_colwidth', None)

In [ ]:
!pip install pyspark

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()
        

In [ ]:
df2 = spark.read.option("header",True).option("parserLib", "UNIVOCITY").option("multiLine", True) \
      .option("delimiter", ",") \
     .csv("/content/sample_data/test.csv", escape='"')
df2.show(50000)    

In [ ]:
df2.count()


In [ ]:
import requests
import csv
with open('/content/sample_data/test2.csv') as csvfile:
    dialect = csv.Sniffer().sniff(csvfile.read(14734))

In [ ]:
csv.field_size_limit()

In [ ]:
import sys, ctypes as ct
csv.field_size_limit(2147483647)
csv.field_size_limit()

In [ ]:
df.count()

In [ ]:
from transformers import AutoTokenizer, TFAutoModel

# model_ckpt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
model_ckpt = "sentence-transformers/all-MiniLM-L12-v2"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

model = TFAutoModel.from_pretrained(model_ckpt, from_pt=True)

In [ ]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

In [ ]:
!pip install transformers

In [ ]:
import torch
def get_embeddings(text_list):
    encoded_input = tokenizer(text_list, padding=True, truncation=True, return_tensors="tf")
    encoded_input = {k: v for k, v in encoded_input.items()}
    with torch.no_grad():
        model_output = model(**encoded_input)
    # return cls_pooling(model_output)
    # return  mean_pooling(model_output, encoded_input["attention_mask"])
    return cls_pooling(model_output)

In [ ]:
import torch
def get_embeddings(text_list):
    encoded_input = tokenizer(text_list, padding=True, truncation=True, return_tensors="tf")
    encoded_input = {k: v for k, v in encoded_input.items()}
    with torch.no_grad():
        model_output = model(**encoded_input)
    # return cls_pooling(model_output)
    # return  mean_pooling(model_output, encoded_input["attention_mask"])
    return cls_pooling(model_output)

In [ ]:
from datasets import Dataset
comments_dataset = Dataset.from_pandas(df)
comments_dataset

In [ ]:
!pip install datasets

In [ ]:
embeddings_dataset= comments_dataset.map(
    lambda x: {"embeddings": get_embeddings(x["abstract"]).numpy()[0]}
)
embeddings_dataset